https://nces.ed.gov/ccd/elsi/tableGenerator.aspx

https://www.usinflationcalculator.com/inflation/current-inflation-rates/
https://data.bls.gov/pdq/SurveyOutputServlet

In [1]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.regression

In [2]:
df = pl.read_csv(
    "./data/nces/geo/ELSI/state/ELSI_csv_export_6385899184786235507585.csv",
    skip_rows=6,
    n_rows=51
).drop("State Name [State] 2018-19").with_columns(
    pl.col("State Name").str.to_titlecase()
)
df = df.unpivot(
    on=df.columns[1:],
    index="State Name",
    variable_name="VARIABLE",
    value_name="VALUE"
).with_columns(
    SCHOOL_YEAR=pl.col("VARIABLE").str.extract(r"(\d{4}\-\d{2})$"),
    VARIABLE=pl.col("VARIABLE").str.replace(r" \[(State( Finance)?)\] (\d{4}\-\d{2})$", ""),
    VALUE=pl.col("VALUE").replace(chr(8211), None).str.to_integer(),
).pivot(
    on="VARIABLE",
    index=["State Name", "SCHOOL_YEAR"],
    values="VALUE",
    aggregate_function="first"
).with_columns(
    pl.sum_horizontal("Kindergarten Students", "Prekindergarten Students", "Grades 1-8 Students", "Grades 9-12 Students").alias("Total Students"),
    pl.sum_horizontal("Grades 1-8 Students", "Grades 9-12 Students").alias("Grades 1-12 Students")
).sort("State Name", "SCHOOL_YEAR").with_columns(
    pl.col("SCHOOL_YEAR").str.extract(r"^(\d{4})").str.to_integer().alias("SCHOOL_YEAR_START"),
    pl.col("SCHOOL_YEAR").str.extract(r"\-(\d{2})$").str.to_integer().add(2000).alias("SCHOOL_YEAR_END")
)
df

State Name,SCHOOL_YEAR,Grades 1-8 Students,Grade 8 Students,Grade 7 Students,Grade 6 Students,Grade 5 Students,Grade 4 Students,Grade 3 Students,Grade 2 Students,Grade 1 Students,Total Expenditures (TE11+E4D+E7A1),Total Revenues (TR) per Pupil (MEMBR),Local Revenues (STR1+R2) per Pupil (MEMBR),State Revenues (R3) per Pupil (MEMBR),Federal Revenues (STR4) per Pupil (MEMBR),Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR),Grades 9-12 Students,Prekindergarten Students,Kindergarten Students,Total Students,Grades 1-12 Students,SCHOOL_YEAR_START,SCHOOL_YEAR_END
str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Alabama""","""2011-12""",461122,57919,59050,59082,58415,56491,55991,56315,57859,7229298957,9534,3125,5284,1126,9709,217615,8282,57602,744621,678737,2011,2012
"""Alabama""","""2012-13""",458974,58151,59685,58240,56523,55859,55868,56361,58287,7438871954,9653,3225,5286,1142,9990,217203,9032,59428,744637,676177,2012,2013
"""Alabama""","""2013-14""",458327,58799,58455,56428,55699,55719,56003,56909,60315,7591336635,9913,3341,5448,1124,10173,218705,9415,59757,746204,677032,2013,2014
"""Alabama""","""2014-15""",454081,57471,56251,54859,55265,55479,56007,58370,60379,7616860366,9992,3321,5549,1122,10235,221068,11076,57939,744164,675149,2014,2015
"""Alabama""","""2015-16""",452764,55792,55172,54900,55340,55808,57963,58766,59023,7856051026,10200,3485,5578,1138,10562,222182,13230,55613,743789,674946,2015,2016
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Wyoming""","""2014-15""",58774,6927,6903,7181,7076,7438,7588,7835,7826,1929789830,20855,7697,11874,1284,20515,26732,564,7997,94067,85506,2014,2015
"""Wyoming""","""2015-16""",59453,6902,7191,7155,7451,7551,7839,7704,7660,2020050580,21569,7851,12414,1305,21327,26914,509,7841,94717,86367,2015,2016
"""Wyoming""","""2016-17""",58998,7135,7071,7352,7452,7642,7616,7422,7308,1913258414,20508,7128,12122,1258,20317,26924,664,7584,94170,85922,2016,2017


In [3]:
inflation = pl.read_excel(
    "./data/bls/inflation/SeriesReport-20240811173715_e1eda9.xlsx",
    read_options=dict(header_row=11),
)
inflation = inflation.with_columns(
    pl.col("Annual").shift(-1).alias("Annual (Next Year)")
).filter(pl.col("Year") < 2020).sort("Year", descending=True).with_columns(
    (1 + pl.col("Annual (Next Year)") / 100).cum_prod().alias("Current to 2020 Dollar Conversion Rate")
).sort("Year")
inflation

Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual,HALF1,HALF2,Annual (Next Year),Current to 2020 Dollar Conversion Rate
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2010,1.6,1.3,1.1,0.9,0.9,0.9,0.9,0.9,0.8,0.6,0.8,0.8,1.0,1.1,0.8,1.7,1.208257
2011,1.0,1.1,1.2,1.3,1.5,1.6,1.8,2.0,2.0,2.1,2.2,2.2,1.7,1.3,2.0,2.1,1.18806
2012,2.3,2.2,2.3,2.3,2.3,2.2,2.1,1.9,2.0,2.0,1.9,1.9,2.1,2.2,2.0,1.8,1.163624
2013,1.9,2.0,1.9,1.7,1.7,1.6,1.7,1.8,1.7,1.7,1.7,1.7,1.8,1.8,1.7,1.7,1.143049
2014,1.6,1.6,1.7,1.8,2.0,1.9,1.9,1.7,1.7,1.8,1.7,1.6,1.7,1.8,1.7,1.8,1.123942
2015,1.6,1.7,1.8,1.8,1.7,1.8,1.8,1.8,1.9,1.9,2.0,2.1,1.8,1.7,1.9,2.2,1.104069
2016,2.2,2.3,2.2,2.1,2.2,2.2,2.2,2.3,2.2,2.1,2.1,2.2,2.2,2.2,2.2,1.8,1.080302
2017,2.3,2.2,2.0,1.9,1.7,1.7,1.7,1.7,1.7,1.8,1.7,1.8,1.8,2.0,1.7,2.1,1.061201
2018,1.8,1.8,2.1,2.1,2.2,2.3,2.4,2.2,2.2,2.1,2.2,2.2,2.1,2.1,2.2,2.2,1.039374


In [4]:
student_investment = df.join(
    inflation.select("Year", "Current to 2020 Dollar Conversion Rate"),
    left_on="SCHOOL_YEAR_END",
    right_on="Year",
).with_columns(
    (
        pl.col("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR)")
        * pl.col("Current to 2020 Dollar Conversion Rate")
    ).alias("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR) (2020 Dollars)")
).with_columns(
    pl.col("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR) (2020 Dollars)")
    .rolling_sum(4)
    .over("State Name")
    .alias("Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2020 Dollars)"),
).select(
    [
        "State Name",
        "SCHOOL_YEAR",
        "Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2020 Dollars)"
    ]
).filter(
    pl.col("SCHOOL_YEAR").is_in(["2014-15", "2018-19"])
).pivot(
    on="SCHOOL_YEAR",
    index="State Name",
    values="Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2020 Dollars)",
).with_columns(
    (pl.col("2018-19") - pl.col("2014-15")).alias("Investment Change")
).sort("Investment Change")
student_investment

State Name,2014-15,2018-19,Investment Change
str,f64,f64,f64
"""Alaska""",94046.988007,81784.974805,-12262.013202
"""Wyoming""",86562.276805,82196.045171,-4366.231634
"""Louisiana""",54834.153752,52740.849105,-2093.304647
"""West Virginia""",55539.244061,53498.039359,-2041.204702
"""Arkansas""",50161.688322,48975.172334,-1186.515988
…,…,…,…
"""Hawaii""",60234.237008,68145.601147,7911.364139
"""New York""",99396.422089,107649.308984,8252.886895
"""Oregon""",50202.867182,59267.572773,9064.705592


In [5]:
math_performance = pl.concat(
    [
        pl.read_csv(
            "data/nces/geo/nations-report-card/2019/state_grade-8_math_SPCsv202408081556.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2015).alias("Graduating Year")
        ),
        pl.read_csv(
            "data/nces/geo/nations-report-card/2015/state_grade-8_math_SPCsv202408081407.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2019).alias("Graduating Year"),
            pl.col("AP").replace("#", None).str.to_integer()
        ),
    ]
).rename(
    {
        "Jurisdiction": "State Name",
        "MN": "Mean Grade 8 Math Score",
        "AB": "At or Above Basic (%)",
        "AP": "At or Above Proficient (%)",
    }
).pivot(
    on="Graduating Year",
    index="State Name",
    values=["Mean Grade 8 Math Score", "At or Above Basic (%)", "At or Above Proficient (%)"],
).with_columns(
    (pl.col("Mean Grade 8 Math Score_2019") - pl.col("Mean Grade 8 Math Score_2015")).alias("Mean Grade 8 Math Score Change"),
    (pl.col("At or Above Basic (%)_2019") - pl.col("At or Above Basic (%)_2015")).alias("At or Above Basic (%) Change"),
    (pl.col("At or Above Proficient (%)_2019") - pl.col("At or Above Proficient (%)_2015")).alias("At or Above Proficient (%) Change"),
).select(
    "State Name",
    "Mean Grade 8 Math Score Change",
    "At or Above Basic (%) Change",
    "At or Above Proficient (%) Change",
).sort("Mean Grade 8 Math Score Change")

math_performance

State Name,Mean Grade 8 Math Score Change,At or Above Basic (%) Change,At or Above Proficient (%) Change
str,i64,i64,i64
"""District of Columbia""",-6,-4,-4
"""Florida""",-4,-2,-5
"""Louisiana""",-4,-4,-5
"""North Carolina""",-3,-2,-4
"""Mississippi""",-3,-2,-2
…,…,…,…
"""Texas""",4,7,2
"""Hawaii""",4,5,2
"""Rhode Island""",5,8,3


In [6]:
__df = student_investment.join(
    math_performance,
    on="State Name"
).sort("Investment Change")

for exog in [
    "Mean Grade 8 Math Score Change",
    "At or Above Basic (%) Change",
    "At or Above Proficient (%) Change",
]:
    result = statsmodels.regression.linear_model.OLS(
        __df["Investment Change"].to_list(),
        __df[exog].to_list(),
    ).fit()
    print("-" * 10, "Investment Change ~", exog, "-" * 10)
    display(result.summary())


---------- Investment Change ~ Mean Grade 8 Math Score Change ----------


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.024
Model:                            OLS   Adj. R-squared (uncentered):              0.004
Method:                 Least Squares   F-statistic:                              1.220
Date:                Tue, 13 Aug 2024   Prob (F-statistic):                       0.275
Time:                        00:22:24   Log-Likelihood:                         -489.34
No. Observations:                  50   AIC:                                      980.7
Df Residuals:                      49   BIC:                                      982.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1           260.2126    235.568      1.105      0.275    -213.178     733.603
==============================================================================
Omnibus:                       17.477   Durbin-Watson:                   0.159
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.401
Skew:                          -0.760   Prob(JB):                     3.09e-11
Kurtosis:                       7.574   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

---------- Investment Change ~ At or Above Basic (%) Change ----------


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.152
Model:                            OLS   Adj. R-squared (uncentered):              0.135
Method:                 Least Squares   F-statistic:                              8.805
Date:                Tue, 13 Aug 2024   Prob (F-statistic):                     0.00464
Time:                        00:22:24   Log-Likelihood:                         -485.82
No. Observations:                  50   AIC:                                      973.6
Df Residuals:                      49   BIC:                                      975.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1           452.1560    152.382      2.967      0.005     145.933     758.379
==============================================================================
Omnibus:                       23.942   Durbin-Watson:                   0.320
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               76.623
Skew:                          -1.100   Prob(JB):                     2.30e-17
Kurtosis:                       8.652   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

---------- Investment Change ~ At or Above Proficient (%) Change ----------


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):             -0.019
Method:                 Least Squares   F-statistic:                            0.04891
Date:                Tue, 13 Aug 2024   Prob (F-statistic):                       0.826
Time:                        00:22:24   Log-Likelihood:                         -489.93
No. Observations:                  50   AIC:                                      981.9
Df Residuals:                      49   BIC:                                      983.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            47.1097    213.012      0.221      0.826    -380.953     475.173
==============================================================================
Omnibus:                       11.988   Durbin-Watson:                   0.088
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               26.879
Skew:                          -0.479   Prob(JB):                     1.46e-06
Kurtosis:                       6.462   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
def generate_state_investment_and_math_performance_bar_chart(
    column: str, legend_title: str, description: str
) -> go.Figure:
    return px.bar(
        student_investment.join(
            math_performance,
            on="State Name"
        ).sort("Investment Change"),
        x="State Name",
        y="Investment Change",
        color=column,
        title=f"US Primary Education Return on Investment<br><sup><i>{description}</i></sup>",
        template="plotly_dark",
        color_continuous_scale=px.colors.sequential.RdBu,
        labels={
            "State Name": "State",
            "Investment Change": "4-Year Expenditure Change (2020 Dollars)",
            column: f"Change in NAEP {legend_title}",
        },
        width=1000,
        height=600,
    ).update_layout(
        xaxis=dict(showticklabels=False, title=None),
        yaxis_title="Per Pupil, 4-Year Expenditure Change Between 2015 and 2019<br>(2020 Dollars)",
        yaxis_title_font_size=10,
        coloraxis_colorbar_title=f"Change in <i>NAEP {legend_title}</i>",
        coloraxis_colorbar_orientation="h",
        plot_bgcolor="#171717",
        paper_bgcolor="#171717",
    ).add_annotation(
        x=1.07,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Chart by Dominic Tarro | 𝕏 @dominictarro",
                "Sources: NAEP Mathematics Assessments (2015, 2019)",
                "CCD National Public Education Financial Survey (2010-2019)"
            )
        ),
        align="right",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    ).add_annotation(
        x=-0.08,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "The NAEP Mathematics scale ranges from 0 to 500.",
                "2020 dollar adjustment for school year expenditures was done with the ending year's inflation rate.",
                "Data compares differences in scores and spending, by state, between 8th graders in 2015 and 2019.",
                "<i>Per Pupil, 4-Year Expenditure</i> is an aggregate from 5th-8th grade."
            )
        ),
        align="left",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    )

generate_state_investment_and_math_performance_bar_chart(
    "Mean Grade 8 Math Score Change",
    "Average Math Score",
    "States spending more per student from 5th through 8th grade had a negligible relationship with average 8th grade math performance."
).show()
generate_state_investment_and_math_performance_bar_chart(
    "At or Above Basic (%) Change",
    "At or Above Basic</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a small relationship with basic 8th grade math performance."
).show()
generate_state_investment_and_math_performance_bar_chart(
    "At or Above Proficient (%) Change",
    "At or Above Proficient</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a negligible relationship with 8th grade math profieciency."
).show()

In [8]:
reading_performance = pl.concat(
    [
        pl.read_csv(
            "data/nces/geo/nations-report-card/2019/state_grade-8_reading_SPCsv202408081536.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2015).alias("Graduating Year")
        ),
        pl.read_csv(
            "data/nces/geo/nations-report-card/2015/state_grade-8_reading_SPCsv202408081449.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2019).alias("Graduating Year"),
        ),
    ]
).with_columns(
            pl.col("MN").replace(chr(8212), None).str.to_integer(),
            pl.col("AB").replace(chr(8212), None).str.to_integer(),
            pl.col("AP").replace(chr(8212), None).str.to_integer(),

).rename(
    {
        "Jurisdiction": "State Name",
        "MN": "Mean Grade 8 Reading Score",
        "AB": "At or Above Basic (%)",
        "AP": "At or Above Proficient (%)",
    }
).pivot(
    on="Graduating Year",
    index="State Name",
    values=["Mean Grade 8 Reading Score", "At or Above Basic (%)", "At or Above Proficient (%)"],
).with_columns(
    (pl.col("Mean Grade 8 Reading Score_2019") - pl.col("Mean Grade 8 Reading Score_2015")).alias("Mean Grade 8 Reading Score Change"),
    (pl.col("At or Above Basic (%)_2019") - pl.col("At or Above Basic (%)_2015")).alias("At or Above Basic (%) Change"),
    (pl.col("At or Above Proficient (%)_2019") - pl.col("At or Above Proficient (%)_2015")).alias("At or Above Proficient (%) Change"),
).select(
    "State Name",
    "Mean Grade 8 Reading Score Change",
    "At or Above Basic (%) Change",
    "At or Above Proficient (%) Change",
).sort("Mean Grade 8 Reading Score Change")

reading_performance

State Name,Mean Grade 8 Reading Score Change,At or Above Basic (%) Change,At or Above Proficient (%) Change
str,i64,i64,i64
"""Puerto Rico""",null,null,null
"""Mississippi""",-4,-4,-5
"""DoDEA""",-3,-1,-5
"""North Carolina""",-2,0,-3
"""Louisiana""",-2,-2,-4
…,…,…,…
"""Minnesota""",6,7,6
"""Iowa""",6,8,3
"""Alabama""",6,7,2


In [9]:
__df = student_investment.join(
    reading_performance,
    on="State Name"
).sort("Investment Change")

for exog in [
    "Mean Grade 8 Reading Score Change",
    "At or Above Basic (%) Change",
    "At or Above Proficient (%) Change",
]:
    result = statsmodels.regression.linear_model.OLS(
        __df["Investment Change"].to_list(),
        __df[exog].to_list(),
    ).fit()
    print("-" * 10, "Investment Change ~", exog, "-" * 10)
    display(result.summary())


---------- Investment Change ~ Mean Grade 8 Reading Score Change ----------


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.063
Model:                            OLS   Adj. R-squared (uncentered):              0.044
Method:                 Least Squares   F-statistic:                              3.303
Date:                Tue, 13 Aug 2024   Prob (F-statistic):                      0.0753
Time:                        00:22:25   Log-Likelihood:                         -488.32
No. Observations:                  50   AIC:                                      978.6
Df Residuals:                      49   BIC:                                      980.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1           295.7873    162.744      1.817      0.075     -31.259     622.834
==============================================================================
Omnibus:                       14.945   Durbin-Watson:                   0.179
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               39.123
Skew:                          -0.615   Prob(JB):                     3.20e-09
Kurtosis:                       7.155   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

---------- Investment Change ~ At or Above Basic (%) Change ----------


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.123
Model:                            OLS   Adj. R-squared (uncentered):              0.105
Method:                 Least Squares   F-statistic:                              6.847
Date:                Tue, 13 Aug 2024   Prob (F-statistic):                      0.0118
Time:                        00:22:25   Log-Likelihood:                         -486.68
No. Observations:                  50   AIC:                                      975.4
Df Residuals:                      49   BIC:                                      977.3
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1           336.3364    128.532      2.617      0.012      78.041     594.631
==============================================================================
Omnibus:                       14.623   Durbin-Watson:                   0.195
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               35.256
Skew:                          -0.639   Prob(JB):                     2.21e-08
Kurtosis:                       6.910   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

---------- Investment Change ~ At or Above Proficient (%) Change ----------


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.001
Model:                            OLS   Adj. R-squared (uncentered):             -0.020
Method:                 Least Squares   F-statistic:                            0.02618
Date:                Tue, 13 Aug 2024   Prob (F-statistic):                       0.872
Time:                        00:22:25   Log-Likelihood:                         -489.94
No. Observations:                  50   AIC:                                      981.9
Df Residuals:                      49   BIC:                                      983.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            34.5161    213.311      0.162      0.872    -394.148     463.180
==============================================================================
Omnibus:                       12.086   Durbin-Watson:                   0.087
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               26.975
Skew:                          -0.490   Prob(JB):                     1.39e-06
Kurtosis:                       6.462   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
def generate_state_investment_and_reading_performance_bar_chart(
    column: str, legend_title: str, description: str
) -> go.Figure:
    return px.bar(
        student_investment.join(
            reading_performance,
            on="State Name"
        ).sort("Investment Change"),
        x="State Name",
        y="Investment Change",
        color=column,
        title=f"US Primary Education Return on Investment<br><sup><i>{description}</i></sup>",
        template="plotly_dark",
        color_continuous_scale=px.colors.sequential.RdBu,
        labels={
            "State Name": "State",
            "Investment Change": "4-Year Expenditure Change (2020 Dollars)",
            column: f"Change in NAEP {legend_title}",
        },
        width=1000,
        height=600,
    ).update_layout(
        xaxis=dict(showticklabels=False, title=None),
        yaxis_title="Per Pupil, 4-Year Expenditure Change Between 2015 and 2019<br>(2020 Dollars)",
        yaxis_title_font_size=10,
        coloraxis_colorbar_title=f"Change in <i>NAEP {legend_title}</i>",
        coloraxis_colorbar_orientation="h",
        plot_bgcolor="#171717",
        paper_bgcolor="#171717",
    ).add_annotation(
        x=1.07,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Chart by Dominic Tarro | 𝕏 @dominictarro",
                "Sources: NAEP Reading Assessments (2015, 2019)",
                "CCD National Public Education Financial Survey (2010-2019)"
            )
        ),
        align="right",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    ).add_annotation(
        x=-0.08,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "The NAEP Reading scale ranges from 0 to 500.",
                "2020 dollar adjustment for school year expenditures was done with the ending year's inflation rate.",
                "Data compares differences in scores and spending, by state, between 8th graders in 2015 and 2019.",
                "<i>Per Pupil, 4-Year Expenditure</i> is an aggregate from 5th-8th grade."
            )
        ),
        align="left",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    )

generate_state_investment_and_reading_performance_bar_chart(
    "Mean Grade 8 Reading Score Change",
    "Average Reading Score",
    "States spending more per student from 5th through 8th grade had a negligible relationship with average 8th grade reading performance."
).show()
generate_state_investment_and_reading_performance_bar_chart(
    "At or Above Basic (%) Change",
    "At or Above Basic</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a small relationship with basic 8th grade reading performance."
).show()
generate_state_investment_and_reading_performance_bar_chart(
    "At or Above Proficient (%) Change",
    "At or Above Proficient</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a negligible relationship with 8th grade reading proficiency."
).show()

In [11]:
"${:,.2f}".format(
    student_investment.join(
        df.filter(pl.col("SCHOOL_YEAR") == "2018-19").select("State Name", "Grade 8 Students"),
        on="State Name"
    ).with_columns(
        (pl.col("Investment Change") * pl.col("Grade 8 Students")).alias("Total Investment Change"),
    )["Total Investment Change"].sum()
)

'$14,141,710,573.34'

In [12]:
math_performance["Mean Grade 8 Math Score Change"].mean(), reading_performance["Mean Grade 8 Reading Score Change"].mean()

(0.6296296296296297, 2.509433962264151)

In [13]:
math_performance["At or Above Basic (%) Change"].mean(), reading_performance["At or Above Basic (%) Change"].mean()

(2.3518518518518516, 3.6226415094339623)

In [14]:
math_performance["At or Above Proficient (%) Change"].mean(), reading_performance["At or Above Proficient (%) Change"].mean()

(-0.32075471698113206, 0.6415094339622641)